In [618]:
from collections import namedtuple

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [619]:
class Task:
    Parameters = namedtuple('Parameters', 'O C D T A')

    def __init__(self, parameters) -> None:
        self.parameters = parameters

    def __str__(self) -> str:
        return str(self.parameters)


In [620]:
filename = 'data/example.txt'
tasks = []

with open(filename) as fp:
    n = fp.readline()
    n = int(n)

    for i in range(n):
        line = fp.readline()
        line = line.strip()
        params = [int(v) for v in line.split()]

        task = Task( Task.Parameters(*params) )
        tasks.append(task)

    

In [621]:
def compute_H(tasks):
    periods = [task.parameters.T for task in tasks]
    H = np.lcm.reduce(periods)
    return H

In [769]:
def get_ticks(tasks, for_deadlines=False, for_dummy=False):
    H = compute_H(tasks) if tasks else 0

    ticks = []
    ids = []
    if for_dummy:
        ticks = list( range(H) )
        ids = [0] * H
        tasks = []
    
    for i, task in enumerate(tasks):
        offset = task.parameters.O
        upper = H
        if for_deadlines:
            offset += task.parameters.D
            upper += task.parameters.D

        _ticks = list( range(offset, upper, task.parameters.T) )
        ticks += _ticks
        ids += [i + 1] * len(_ticks)

    df_ticks = pd.DataFrame( zip(ticks, ids), columns=['t', 'ids'])
    df_ticks['size'] = 1
    df_ticks['r'] = 60
    df_ticks['symbol'] = 'x-thin-open'

    if for_deadlines:
        df_ticks['r'] = 50
        df_ticks['symbol'] = 'circle-x'
    elif for_dummy:
        df_ticks['size'] = 0

    df_ticks['ids'] = df_ticks['ids'].astype(str)

    # Format ticks
    df_ticks['t'] = df_ticks['t'] * (360 / H) % 360
    # df_ticks['t'] = df_ticks['t'].astype(str)
    
    return df_ticks

In [779]:
def plot_events(df_ticks, size=600):
    fig = px.scatter_polar(df_ticks, r='r', theta='t', color='ids', size='size', 
                        symbol='symbol', symbol_map='identity',
                        start_angle=0, width=size, height=size,
                        direction='counterclockwise', color_discrete_sequence=px.colors.qualitative.Alphabet)
    return fig

In [780]:
df_ticks_releases = get_ticks(tasks)
df_ticks_deadlines = get_ticks(tasks, True)
df_ticks_dummy = get_ticks(tasks, for_dummy=True)

In [781]:
fig_releases = plot_events(df_ticks_releases)
fig_deadlines = plot_events(df_ticks_deadlines)

In [871]:
H = compute_H(tasks)
width = 360 / H
theta = [(t * width) for t in range(H)]
color = [t % 3 for t in range(H)]

df_t = pd.DataFrame( zip(theta, color), columns=['theta', 'color'])
df_t['color'] = df_t['color'].astype(str)

df_t['r'] = 50
df_t['width'] = width


In [891]:
fig_trace = px.bar_polar(df_t, r='r', theta='theta', color='color',
                        start_angle=0, width=600, height=600,
                        direction='counterclockwise', color_discrete_sequence=px.colors.qualitative.Alphabet)
fig_trace.show()

In [892]:
d = fig_trace.data[0]

In [893]:
# d?

In [894]:
# go.barpolar.Marker?

In [895]:
df_t

,theta,color,r,width
0,0.0,0,50,20.0
1,20.0,1,50,20.0
2,40.0,2,50,20.0
3,60.0,0,50,20.0
4,80.0,1,50,20.0
5,100.0,2,50,20.0
6,120.0,0,50,20.0
7,140.0,1,50,20.0
8,160.0,2,50,20.0
9,180.0,0,50,20.0


In [896]:
bar = go.Barpolar(
    r=df_t['r'],
    theta=df_t['theta'],
    width=df_t['width'],
    marker_color=px.colors.qualitative.Alphabet[i],
    marker_line_color='white',
    marker_line_width=0.2,
    opacity=0.8
)

In [913]:
fig = plot_events(df_ticks_dummy)
for i in range(len(tasks)):
    fig.add_trace(fig_releases.data[i])
    fig.add_trace(fig_deadlines.data[i])

fig.add_trace(bar)

fig.update_layout(
    template=None,
    polar = dict(
        # angularaxis = dict(ticks='outside', tickvals=[str(t) for t in range(H)]),
        radialaxis = dict(showticklabels=False, ticks=''),
    )
)
fig.show()

In [785]:
# px.bar_polar?

In [826]:
fig = go.Figure(go.Barpolar(
    r=df_t['r'],
    theta=df_t['theta'],
    width=df_t['width'],
    marker_color=px.colors.qualitative.Alphabet,
    # marker_color=["#E4FF87", '#709BFF', '#709BFF', '#FFAA70', '#FFAA70', '#FFDF70', '#B6FFB4'],
    marker_line_color='white',
    marker_line_width=0.2,
    opacity=0.8
))

fig.update_layout(
    template=None,
    polar = dict(
        radialaxis = dict(range=[0, 5], showticklabels=False, ticks=''),
        angularaxis = dict(showticklabels=False, ticks='')
    )
)

fig.show()

In [831]:
fig = go.Figure(go.Barpolar(
    r=df_t['r'],
    theta=df_t['theta'],
    width=df_t['width'],
    marker_color=px.colors.qualitative.Alphabet,
    # marker_color=["#E4FF87", '#709BFF', '#709BFF', '#FFAA70', '#FFAA70', '#FFDF70', '#B6FFB4'],
    marker_line_color='white',
    marker_line_width=0.2,
    opacity=0.8
))

fig.update_layout(
    template=None,
    polar = dict(
        # radialaxis = dict(range=[0, 5], showticklabels=False, ticks=''),
        angularaxis = dict(showticklabels=False, ticks='')
    )
)

fig.show()